<a href="https://colab.research.google.com/github/nmach22/Promoter-Classification/blob/main/notebooks/train_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Set Env**

In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/nmach22/Promoter-Classification.git
!pip install -r ./Promoter-Classification/requirements.txt


# **Imports**

In [37]:

# Imports
import sys
import os
import torch
import yaml
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid

# Add the root directory of the cloned repository to the Python path
ROOT_DIR = '/content/Promoter-Classification'
sys.path.append(ROOT_DIR)


import importlib
import utils.fasta_dataset as dataset_module
import utils.encoding_functions as encoding_module
importlib.reload(dataset_module)
importlib.reload(encoding_module)
from utils.fasta_dataset import FastaDataset
from utils.encoding_functions import *

with open(f"{ROOT_DIR}/config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

print(config.keys())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

dict_keys(['data'])
cpu


In [43]:
ecoli_data = config['data']['ecoli']

prom_path = f"{ROOT_DIR}/{ecoli_data['promoter_fasta']}"
non_prom_path = f"{ROOT_DIR}/{ecoli_data['non_promoter_fasta']}"

dataset = FastaDataset(prom_path, non_prom_path, seq_len=config['data']['ecoli']['seq_len'])

In [19]:
len(dataset.sequences[4])

81

In [40]:
x, y = dataset[0]

In [45]:
len(dataset)

3839